In [1]:
adj = readdlm("Data/Pokec/pokec-_edges.txt");

lab_region = readdlm("Data/Pokec/pokec-region_labels.txt");#lab_gender = readdlm("Data/Pokec/pokec-gender_labels.txt");

community_true0 = lab_region[:,2];

N = length(community_true0);
n_edge0 = size(adj)[1];

A0 = sparse(adj[:,1],adj[:,2],vec(ones(Int64,n_edge0,1)),N,N);

lab_gender = 0;lab_region =0;community_true0 =0;
adj = 0;
gc();

A = A0+A0';

N = size(A)[1];
n_edge0 = nnz(A)/2;

A0 =0;p=0;
gc();

In [2]:
d = sum(A,2);
s = sum(d);

using Distributions.Uniform
r = 70;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[239.113, 179.002, 175.91, 160.602, 156.335, 147.208, 143.448, 141.341, 140.773, 139.57, 135.431, 131.795, 126.265, 125.947, 125.259, 124.919, 124.073, 124.004, 123.359, 121.077, 119.982, 119.568, 118.314, 117.323, 117.382, 115.293, 115.06, 114.341, 114.142, 113.53, 113.065, 112.992, 111.73, 111.255, 110.648, 110.621, 110.485, 110.232, 109.453, 108.711, 108.485, 108.188, 108.011, 107.617, 107.32, 106.983, 106.741, 106.484, 105.231, 104.832, 104.536, 104.412, 104.095, 103.465, 103.382, 103.046, 102.905, 102.611, 102.345, 102.309, 101.701, 101.535, 101.008, 100.924, 100.72, 100.417, 100.118, 100.279, 99.367, 99.3817]

In [14]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:52]';

n_clusters = 200;
n_it_Kmeans = 10;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;

# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 2
Number of communities: 44
Modularity: 0.616429375700194


In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 100;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:70
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best 
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


LoadError: [91mInterruptException:[39m